## テーマ

リニアSVMとロジスティック回帰を使い二項分類において両アルゴリズムを比較する。


## 目的

リニアSVMとロジスティック回帰はともにデータを複数のカテゴリに分類することができるアルゴリズムである。
両アルゴリズムにおいて、分類結果にどのような違いが生じるのかを実験を行い、
両アルゴリズムがどのような場合に適しているのかを考察する。

## データセット

kaggleの[Titanic competition](https://www.kaggle.com/c/titanic/data)のデータを使用

## imports

In [83]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

## loading data

| variable | definition |
--- | --- 
| survival | 生存(0 = No, 1 = Yes) |
| pclass | Ticket class(1 = 1st, 2 = 2nd, 3 = 3rd) |
| sex | 性別 |
| Age | 年齢 |
| Name | 名前 |
| sibsp | 同乗の兄弟・配偶者 |
| parch | 同乗の親・子供 |
| ticket | チケットID|
| fare | 運賃 |
| cabin | 部屋番号 |
| embarked | 乗船港 |

In [77]:
train = pd.read_csv('./titanic/train.csv')
print(train.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [34]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


## 特徴量の調査

In [53]:
print(train.isnull().any())

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool


Age,Cabin,Embarkedは欠損値が存在しているので、何かしらのデータを代入する。

In [54]:
print(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


7つは整数または浮動小数点で、5つは文字列となっている。

In [56]:
# numerical

numerical_train = train[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
print(numerical_train.describe())

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


- サンプル数は891です。
- 生存率は38％です。
- 25%の乗客は兄弟や配偶者と一緒でした
- 運賃には大きな差があります

In [60]:
# object

object_train = train[['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']]
print(object_train.describe())

                           Name   Sex  Ticket    Cabin Embarked
count                       891   891     891      204      889
unique                      891     2     681      147        3
top     Braund, Mr. Owen Harris  male  347082  B96 B98        S
freq                          1   577       7        4      644


- 名前は一意
- 男性が65%
- チケットは22%が重複している
- 乗船港は3種類あり、ほとんどがS


## カラムの削除

- ticket: 重複率が高いので削除
- cabin: 欠損値が多く削除
- PassengerId: IDなので削除
- Name: 名前が生存率に影響するとは考えにくいので削除


In [78]:
train = train.drop(['Ticket', 'Cabin', 'PassengerId', 'Name'], axis=1)

## 欠損値の穴埋め

最頻値や平均値で穴埋めを行う

In [79]:
embarke_freq = train.Embarked.dropna().mode()[0]
age_mean = train.Age.dropna().mean()

train['Embarked'] = train['Embarked'].fillna(embarke_freq)
train['Age'] = train['Age'].fillna(age_mean)

## カテゴリデータを数値に変換

In [80]:
train['Sex'] = train['Sex'].map({'female': 1, 'male': 0}).astype(int)
train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
print(train.head())

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0         0       3    0  22.0      1      0   7.2500         0
1         1       1    1  38.0      1      0  71.2833         1
2         1       3    1  26.0      0      0   7.9250         0
3         1       1    1  35.0      1      0  53.1000         0
4         0       3    0  35.0      0      0   8.0500         0


## ラベルと特徴量に分離

In [81]:
x_train = train.drop('Survived', axis=1)
y_train = train['Survived']

## ロジスティクス回帰のモデル作成

In [92]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
score = round(logreg.score(x_train, y_train) * 100, 2)
print(score)

coeff_df = pd.DataFrame(train.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df['Correlation'] = pd.Series(logreg.coef_[0])

print(coeff_df.sort_values(by='Correlation', ascending=False))

80.47
    Feature  Correlation
1       Sex     2.607417
6  Embarked     0.256137
5      Fare     0.002372
2       Age    -0.039196
4     Parch    -0.078888
3     SibSp    -0.319913
0    Pclass    -1.086917


## リニアSVMのモデルを作成

In [97]:
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
score_svc = round(linear_svc.score(x_train, y_train) * 100, 2)
print(score_svc)

coeff_df_svc = pd.DataFrame(train.columns.delete(0))
coeff_df_svc.columns = ['Feature']
coeff_df_svc['Correlation'] = pd.Series(linear_svc.coef_[0])

print(coeff_df_svc.sort_values(by='Correlation', ascending=False))


79.24
    Feature  Correlation
1       Sex     0.948564
6  Embarked     0.127388
4     Parch     0.047889
5      Fare    -0.002112
2       Age    -0.020474
3     SibSp    -0.152121
0    Pclass    -0.356814


/Users/hiroki-ikeda/my-lab/study/jaist-study-plan/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## 考察

ロジスティック回帰とリニアSVMを同様の教師データを元にモデルを作成した。その結果を考察する。
両アルゴリズムのモデルの教師データの結果と推論値の正解率が大きな差は無い。

各特徴量の重みについては差分がでた。
重みを降順に並べたときに順番が違うが、他の特徴量と比べ相関の弱い特徴について差分が出ている。
性別については両アルゴリズムでともに最も重いが、他のその重みについては大きな差が出ている。

ロジスティック回帰は特徴量の中で相関の強い特徴があったときに、その特徴の重みに大きく依存するモデルとなった。
リニアSVMは特定の特徴量に強く依存するようなモデルにはならなかった。

一概に言えることではないが、特徴量が多い場合にはリニアSVMを使い、少ない場合にはロジスティック回帰を使うことでモデルを最適化できる。